# Implement webmaps with data downloads on District Digest
* Issue #1559
* Reference `ca_transit_speed_maps/05_webmap_example`
* Maps are here: https://console.cloud.google/storage/browser/calitp-map-tiles/caltrans_district_digest?pageState=(%22StorageObjectListTable%22:(%22f%22:%22%255B%255D%22))&csesidx=1815466960&project=cal-itp-data-infra

In [1]:
import branca
import geopandas as gpd
import numpy as np
import pandas as pd
import _ct_district_data_prep

from calitp_data_analysis import geography_utils
from calitp_data_analysis.gcs_geopandas import GCSGeoPandas
from shared_utils import catalog_utils, webmap_utils

gcsgp = GCSGeoPandas()

In [2]:
district = "03 - Marysville / Sacramento"

In [3]:
district_int = [int(s) for s in district.split() if s.isdigit()][0]

In [4]:
# Load Datasets
operator_df = _ct_district_data_prep.data_wrangling_operator_profile(district)

In [5]:
operator_gdf = _ct_district_data_prep.data_wrangling_operator_map(
    list(operator_df.portfolio_organization_name.unique())
)

In [6]:
ct_district_gdf = _ct_district_data_prep.load_ct_district(district_int)

In [7]:
transit_route_shs_gdf, transit_route_shs_table = (
    _ct_district_data_prep.final_transit_route_shs_outputs(15, district_int)
)
district_gdf = _ct_district_data_prep.load_ct_district(district_int)
gtfs_table_df = _ct_district_data_prep.create_gtfs_stats(operator_df)
shn_gdf = _ct_district_data_prep.load_buffered_shn_map(50, district_int)

In [8]:
len(operator_gdf)

16

## Transit Routes  V1 = District at the bottom
* The district map completely covers the transit routes.

In [9]:
cmap = branca.colormap.linear.PuBu_09.scale()

In [10]:
cmap.colors

[(1.0, 0.9686274509803922, 0.984313725490196, 1.0),
 (0.9254901960784314, 0.9058823529411765, 0.9490196078431372, 1.0),
 (0.8156862745098039, 0.8196078431372549, 0.9019607843137255, 1.0),
 (0.6509803921568628, 0.7411764705882353, 0.8588235294117647, 1.0),
 (0.4549019607843137, 0.6627450980392157, 0.8117647058823529, 1.0),
 (0.21176470588235294, 0.5647058823529412, 0.7529411764705882, 1.0),
 (0.0196078431372549, 0.4392156862745098, 0.6901960784313725, 1.0),
 (0.01568627450980392, 0.35294117647058826, 0.5529411764705883, 1.0),
 (0.00784313725490196, 0.2196078431372549, 0.34509803921568627, 1.0)]

In [11]:
cmap_v2 = branca.colormap.linear.YlOrBr_09.scale()

In [12]:
cmap_v2

In [13]:
operator_gdf.shape

(16, 3)

In [14]:
cmap_all_routes = branca.colormap.LinearColormap(
    colors=cmap_v2.colors[4:],
    vmin=operator_gdf.number.min(),
    vmax=operator_gdf.number.max(),
)

In [15]:
cmap_all_routes

In [16]:
district_map = webmap_utils.set_state_export(
    district_gdf,
    filename="test_cc_routes1",
    map_title="District Map",
)

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:118: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


In [17]:
transit_routes = webmap_utils.set_state_export(
    operator_gdf,
    filename="test2",
    map_title="Transit Routes",
    cmap=cmap_all_routes,
    color_col="number",
    existing_state=district_map
    
)

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:118: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


In [18]:
webmap_utils.render_spa_link(transit_routes["spa_link"], text="Test Layered Map")

<a href="https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJEaXN0cmljdCBNYXAiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL3Rlc3RpbmcvdGVzdF9jY19yb3V0ZXMxLmdlb2pzb24uZ3oiLCAicHJvcGVydGllcyI6IHsic3Ryb2tlZCI6IGZhbHNlLCAiaGlnaGxpZ2h0X3NhdHVyYXRpb25fbXVsdGlwbGllciI6IDAuNX19LCB7Im5hbWUiOiAiVHJhbnNpdCBSb3V0ZXMiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL3Rlc3RpbmcvdGVzdDIuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41fX1dLCAibGF0X2xvbiI6IFszOC44NzA3NTEwMjMxNTExODYsIC0xMTkuNTQ1MTg1NjMxOTIxXSwgInpvb20iOiAxM30=" target="_blank">Open Test Layered Map in New Tab</a>

In [19]:
webmap_utils.display_spa_map(transit_routes["spa_link"])

## Transit Routes V2
* Putting the transit routes at the top
* It looks exactly the same. 

In [20]:
transit_routes_v2 = webmap_utils.set_state_export(
    operator_gdf,
    filename="test2",
    map_title="Transit Routes",
    cmap=cmap_all_routes,
    color_col="number",
    
)

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:118: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


In [21]:
district_map_v2 = webmap_utils.set_state_export(
    district_gdf,
    filename="test_cc_routes1",
    map_title="District Map",
    existing_state=transit_routes_v2
)

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:118: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


In [22]:
webmap_utils.render_spa_link(transit_routes["spa_link"], text="Test Layered Map 2")

<a href="https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJEaXN0cmljdCBNYXAiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL3Rlc3RpbmcvdGVzdF9jY19yb3V0ZXMxLmdlb2pzb24uZ3oiLCAicHJvcGVydGllcyI6IHsic3Ryb2tlZCI6IGZhbHNlLCAiaGlnaGxpZ2h0X3NhdHVyYXRpb25fbXVsdGlwbGllciI6IDAuNX19LCB7Im5hbWUiOiAiVHJhbnNpdCBSb3V0ZXMiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL3Rlc3RpbmcvdGVzdDIuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41fX1dLCAibGF0X2xvbiI6IFszOC44NzA3NTEwMjMxNTExODYsIC0xMTkuNTQ1MTg1NjMxOTIxXSwgInpvb20iOiAxM30=" target="_blank">Open Test Layered Map 2 in New Tab</a>

## SHS Routes

In [23]:
cmap_shn_routes = branca.colormap.LinearColormap(
    colors=cmap.colors[4:],
    vmin=transit_route_shs_gdf.number.min(),
    vmax=transit_route_shs_gdf.number.max(),
)

In [24]:
cmap_shn_routes

In [25]:
shn_map = webmap_utils.set_state_export(
    shn_gdf,
    filename="test_cc_routes1",
    map_title="State Highway Network Map",
)

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:118: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


In [26]:
webmap_utils.render_spa_link(shn_map["spa_link"], text="SHN Layer")

<a href="https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJTdGF0ZSBIaWdod2F5IE5ldHdvcmsgTWFwIiwgInVybCI6ICJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY2FsaXRwLW1hcC10aWxlcy90ZXN0aW5nL3Rlc3RfY2Nfcm91dGVzMS5nZW9qc29uLmd6IiwgInByb3BlcnRpZXMiOiB7InN0cm9rZWQiOiBmYWxzZSwgImhpZ2hsaWdodF9zYXR1cmF0aW9uX211bHRpcGxpZXIiOiAwLjV9fV0sICJsYXRfbG9uIjogWzM5LjAyMTU2MzgwNjY4MTcyNiwgLTEyMS4zMDA2MTAxNTg4Mzc3XSwgInpvb20iOiAxM30=" target="_blank">Open SHN Layer in New Tab</a>

In [27]:
transit_shn_map = webmap_utils.set_state_export(
    transit_route_shs_gdf,
    filename="test2",
    map_title="Transit Routes on the State Highway Network",
    cmap=cmap,
    color_col="number",
    existing_state=shn_map
    
)

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:118: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


In [28]:
webmap_utils.render_spa_link(transit_shn_map["spa_link"], text="Transit Routes on SHN Layer")

<a href="https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJTdGF0ZSBIaWdod2F5IE5ldHdvcmsgTWFwIiwgInVybCI6ICJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY2FsaXRwLW1hcC10aWxlcy90ZXN0aW5nL3Rlc3RfY2Nfcm91dGVzMS5nZW9qc29uLmd6IiwgInByb3BlcnRpZXMiOiB7InN0cm9rZWQiOiBmYWxzZSwgImhpZ2hsaWdodF9zYXR1cmF0aW9uX211bHRpcGxpZXIiOiAwLjV9fSwgeyJuYW1lIjogIlRyYW5zaXQgUm91dGVzIG9uIHRoZSBTdGF0ZSBIaWdod2F5IE5ldHdvcmsiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL3Rlc3RpbmcvdGVzdDIuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41fX1dLCAibGF0X2xvbiI6IFszOC44NzYxNTIxOTQ1MzA2NSwgLTEyMS4yOTM2MTE3MzY4NjI0Ml0sICJ6b29tIjogMTN9" target="_blank">Open Transit Routes on SHN Layer in New Tab</a>